In [13]:
Today = '30/06/24'
# Today_1 and TODAY_2!!!

Mounth = 'Jun'

print_Debt = True
print_Deriv = True
print_BD = False
excel_tofolder_on_Z = True

In [14]:
import sys
sys.path.append("C:\\Users\\KlimovaAnnaA\\Documents\\MyFiles\\Projects\\OCP")
from Defs import merge_SalesUnits
from Defs import merge_Mapping
from Defs import Period
from Defs import new_list
from Defs import export_from_RISKCUSTOM
from Defs import add_in_currency_column
from Defs import concat_columns
from Defs import export_from_WHWEEK

import pandas as pd
from datetime import date
import numpy as np

In [15]:
query = f"""select * from "RISKACCESS"."quantumDebt" where "reportDate" = TO_DATE('{Today}', 'DD/MM/YY')"""
data_Debt_export = export_from_RISKCUSTOM(query)
data_Debt_export.shape

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


(102706, 73)

In [16]:
data_Debt_export.holding.unique()
max_data = data_Debt_export.effectiveFrom.max()
data_Debt_export.query('effectiveFrom == @max_data').shape

(6855, 73)

Фильтрация данных

In [17]:
max_data = data_Debt_export.effectiveFrom.max() # Присваивание переменной значения мин даты в столбце effectiveFrom

for i in ['SUEK', 'EuroChem']:
    data_Debt_work = data_Debt_export.query('holding == @i & effectiveFrom == @max_data').reset_index(drop=True) # Фильтрация данных по столбцу holding (переменная Group) и effectiveFrom
    data_Debt_work = data_Debt_work[~data_Debt_work.entity.isna()]
    data_Debt_work.entity = data_Debt_work.entity.replace({'RUBND': 'RUSBO'}).reset_index(drop=True)
    if i == 'SUEK':
        data_Debt_work = data_Debt_work.rename(columns={'entity': 'pre_entity'})
        data_Debt_work['entity'] = merge_Mapping(data_Debt_work, col='pre_entity')

    data_Debt_work['Entity_group'] = merge_SalesUnits(data_Debt_work, col='entity', merge_col='ocpSegment').fillna('External')

    # Выполнение цикла для рассчета amount, amountUSD и присваивания значения counterparty для каждого DealSet

    deals_data = data_Debt_work[['entity', 'Entity_group', 'counterparty','amount', 'amountUSD', 'instrumentOwner', 'dealSet']] # Выбор нужного разреза данных
    deals_data_Bonds = deals_data[deals_data.dealSet.str.contains('Bond')] # Фильтрация разреза по значениям в DealSet, которые содержат слово Bond

    Bonds_list = deals_data_Bonds.dealSet.unique()

    for Bond in Bonds_list:
        try:
            Bond_data = deals_data_Bonds.query('dealSet ==@Bond') # Отбор данных одного Bond
            assert len(Bond_data[Bond_data.instrumentOwner == 'EC_ISSUED_BONDS']) != 0

            for column in Bond_data[['amount', 'amountUSD']]:
                Traded_data_sum = Bond_data.query('instrumentOwner =="EC_TRADED_BONDS"')[column].sum()
                Issued_data = Bond_data.query('instrumentOwner =="EC_ISSUED_BONDS"')[column]
                # Внесение нового значения amount в issued bonds
                Bond_data.loc[Bond_data['instrumentOwner'] =="EC_ISSUED_BONDS", column] = Traded_data_sum + Issued_data 
            # Внесение нового значения counterparty в traded bonds
            Issied_data_entity = Bond_data.query('instrumentOwner =="EC_ISSUED_BONDS"').Entity.squeeze()
            Bond_data.loc[Bond_data['instrumentOwner'] =="EC_TRADED_BONDS", 'counterparty'] = Issied_data_entity
            deals_data_Bonds.loc[deals_data_Bonds['dealSet'] == Bond] = Bond_data
        except:
            continue

    deals_data.loc[deals_data.dealSet.str.contains('Bond')] = deals_data_Bonds
    data_Debt_work[['entity', 'Entity_group', 'counterparty','amount', 'amountUSD', 'instrumentOwner', 'dealSet']] = deals_data
    # Запись выполненного цикла в в основные данные

    data_Debt_work = Period(data_Debt_work, day_for_count=Today, col_with_date='actionDate').reset_index(drop=True)

    data_Debt_work.loc[data_Debt_work.counterparty.str.contains('(EC)'), 'counterparty'] = data_Debt_work.loc[data_Debt_work.counterparty.str.contains('(EC)'), 'counterparty'].str[:-5] 
    # Обрезка " (EC)" в каждой строке

    data_Debt_work = data_Debt_work[~data_Debt_work.counterparty.isna()].reset_index(drop=True)
    data_Debt_work['CompCode'] = merge_Mapping(data_Debt_work, col='counterparty').fillna('External')
    data_Debt_work['Counterparty_Group'] = merge_SalesUnits(data_Debt_work, col='CompCode', merge_col='ocpSegment').fillna('External')
    # Merge counterparty с Mapping и SalesUnits

    data_Debt_work['Source'] = 'Quantum'

    if i == 'SUEK':
        data_Debt_work_SUEK = data_Debt_work
    if i == 'EuroChem':
        data_Debt_work_ECH = data_Debt_work

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['SGC' 'SGC' 'SGC' ... 'SUEK RU' 'SUEK RU' 'SPV']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  merge_data.loc[merge_data[f'{id_col}_merge'] != 'External', 'last_merge'] = merge_data.loc[merge_data[f'{id_col}_merge'] != 'External', f'{id_col}_merge']
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_3232\2281321293.py:43: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, us

Запись Debt в Excel:

In [18]:
Sheet_in_output_Debt = 'Debt'
Output_Debt_SUEK = "_".join([str(date.today()), 'SUEK_quantum_Debt', f'{Mounth}.xlsx'])
Output_Debt_ECH = "_".join([str(date.today()), 'Ech_quantum_Debt', f'{Mounth}.xlsx'])
if excel_tofolder_on_Z == True:
    Output_path = 'z:\\Anna_Klimova\\OCP\\Archive\\'
    Output_Debt_SUEK = Output_path + Output_Debt_SUEK
    Output_Debt_ECH = Output_path + Output_Debt_ECH

data_Debt_export_for_print_SUEK = data_Debt_work_SUEK[['entity', 'Entity_group','counterparty',\
                                'Counterparty_Group','amount','eventCurrency','amountUSD','actionDate','Days',\
                                'Period','dealClass1','dealClass2','instrumentOwner','dealSet','facility','Source']]\
                                .rename(columns={'amount': 'amoutn outstanding',\
                                                    'eventCurrency': 'Currency',\
                                                    'amountUSD': 'amount USD eq',\
                                                    'actionDate': 'termEnd'})
data_Debt_export_for_print_ECH = data_Debt_work_ECH[['entity', 'Entity_group','counterparty',\
                                'Counterparty_Group','amount','eventCurrency','amountUSD','actionDate','Days',\
                                'Period','dealClass1','dealClass2','instrumentOwner','dealSet','facility','Source']]\
                                .rename(columns={'amount': 'amoutn outstanding',\
                                                    'eventCurrency': 'Currency',\
                                                    'amountUSD': 'amount USD eq',\
                                                    'actionDate': 'termEnd'})

if print_Debt == True:
    data_Debt_export_for_print_SUEK.to_excel(Output_Debt_SUEK, sheet_name = Sheet_in_output_Debt, index = False)
    data_Debt_export_for_print_ECH.to_excel(Output_Debt_ECH, sheet_name = Sheet_in_output_Debt, index = False) 

In [19]:
check = data_Debt_work[['entity', 'Entity_group', 'counterparty','amount', 'amountUSD', 'instrumentOwner', 'dealSet']]
check = check[check.instrumentOwner.str.contains('BONDS')].reset_index(drop=True)
check[check.dealSet == check.dealSet.unique().tolist()[3]]

,entity,Entity_group,counterparty,amount,amountUSD,instrumentOwner,dealSet
8,RUEMO,RUFERT,KIT Finance Dealing,2100000.0,2100000.0,EC_TRADED_BONDS,SUEK EuBond 2021 $500m B
9,RUEMO,RUFERT,KIT Finance Dealing,1000000.0,1000000.0,EC_TRADED_BONDS,SUEK EuBond 2021 $500m B
10,RUEMO,RUFERT,KIT Finance Dealing,1125000.0,1125000.0,EC_TRADED_BONDS,SUEK EuBond 2021 $500m B
11,RUEMO,RUFERT,KIT Finance Dealing,1900000.0,1900000.0,EC_TRADED_BONDS,SUEK EuBond 2021 $500m B
12,RUEMO,RUFERT,KIT Finance Dealing,200000.0,200000.0,EC_TRADED_BONDS,SUEK EuBond 2021 $500m B
13,RUEMO,RUFERT,KIT Finance Dealing,7750000.0,7750000.0,EC_TRADED_BONDS,SUEK EuBond 2021 $500m B
14,RUEMO,RUFERT,KIT Finance Dealing,939000.0,939000.0,EC_TRADED_BONDS,SUEK EuBond 2021 $500m B
15,RUEMO,RUFERT,KIT Finance Dealing,200000.0,200000.0,EC_TRADED_BONDS,SUEK EuBond 2021 $500m B
16,RUEMO,RUFERT,GENERIC EXTERNAL,200000.0,200000.0,EC_TRADED_BONDS,SUEK EuBond 2021 $500m B
17,RUEMO,RUFERT,KIT Finance Dealing,200000.0,200000.0,EC_TRADED_BONDS,SUEK EuBond 2021 $500m B


In [20]:
manual_map = data_Debt_work.loc[(data_Debt_work['cpartyType'] == 'Internal') & (data_Debt_work['CompCode'] == 'External'), ['counterparty', 'CompCode', 'Counterparty_Group']].drop_duplicates()
manual_map

manual_map2 = data_Debt_work.loc[data_Debt_work['Entity_group'] == 'External', ['entity', 'Entity_group']].drop_duplicates()
manual_map2

manual_map_print = pd.concat([manual_map, manual_map2], axis=1)
manual_map_print
# manual_map_print.to_excel('Manual_map.xlsx', index=False)

,counterparty,CompCode,Counterparty_Group,entity,Entity_group
400,Corrigo,External,External,NaN,NaN


Работа с Forwards и Swaps:

In [21]:
# query = f"""select * from "RISKACCESS"."quantumForwards" """
# data_Forwards_export = export_from_RISKCUSTOM(query)
# data_Forwards_export[data_Forwards_export.positionDate > '2024-03-01'].positionDate.unique()
Today_1 = '2024-06-30'
query = f"""select * from "RISKACCESS"."quantumForwards" where "positionDate" = TO_DATE('{Today_1}', 'YYYY-MM-DD')"""
data_Forwards_export = export_from_RISKCUSTOM(query)

# query = f"""select * from "RISKACCESS"."quantumSwaps" """
# data_Swaps_export = export_from_RISKCUSTOM(query)
# data_Swaps_export[data_Swaps_export.positionDate > '2024-03-01'].positionDate.unique(
Today_2 = '30/06/24'
query = f"""select * from "RISKACCESS"."quantumSwaps" where "positionDate" = TO_DATE('{Today_2}', 'DD/MM/YY')"""
data_Swaps_export = export_from_RISKCUSTOM(query)

data_Forwards_export.shape, data_Swaps_export.shape

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)


((15, 80), (18, 84))

In [22]:
data_work_list = [data_Forwards_export, data_Swaps_export]

for data_Deriv_index in range(len(data_work_list)):
    # Проверка вхождений dealNo в Debt и остановка процесса в случае отсутвия данных:
    data_Deriv = data_work_list[data_Deriv_index]
    dealNo_intersect_list = list(set(data_Deriv['dealNo']).intersection(set(data_Debt_export['dealNo'])))
    data_work_Deriv = data_Deriv[~(data_Deriv.dealNo.isin(dealNo_intersect_list))]
    if len(data_work_Deriv) == 0:
        if data_Deriv_index == 0:
            print('data_Forwards_export is empty')
            data_print_Forwards = pd.DataFrame()
            continue
        else:
            print('data_Swaps_export is empty')
            data_print_Swaps = pd.DataFrame()
            continue
        
    data_work_Deriv['holdingEntity'] = data_work_Deriv['holdingEntity'].replace({'XXTSU': 'SUEK'})

    # Создание входящей пары внизу
    Data_for_loop = data_work_Deriv[['entity','holdingEntity','counterparty','dealNo', 'payFaceValue', 'payFXCurrency', 'recFaceValue', 'recFXCurrency', 'maturityDate']]
    for i in range(0, len(Data_for_loop)):
        row_list = list(Data_for_loop.loc[i])
        Data_for_loop.loc[len(Data_for_loop)] = row_list[:4] + row_list[6:8] + row_list[6:]
        Data_for_loop.loc[i,'payFaceValue'] = Data_for_loop.loc[i,'payFaceValue'] * -1
    Data_for_loop = Data_for_loop.sort_values('dealNo')
    # цикл создания новых строк
    Data_for_loop = Period(Data_for_loop, day_for_count=Today_1, col_with_date='maturityDate')
    # Создание Days и Period
    Data_for_loop = add_in_currency_column(Data_for_loop, CCY_to='USD', col_with_CCY='payFXCurrency', date_is_column=False, col_with_VAL='payFaceValue', DATE=Today_1)
    # создание payFaceValue_in_USD
    new_columns = ['Company', 'Settlement', 'Notional_Amount_(USD)', 'Unnamed', 'source']
    Data_for_loop = Data_for_loop.reindex(columns=(Data_for_loop.columns.tolist() + new_columns))
    Data_for_loop['source'] = 'Quantum'
    # Создание пустых столбцов и столбца ресурса

    if data_Deriv_index == 0:
        data_print_Forwards = Data_for_loop
    else:
        data_print_Swaps = Data_for_loop

data_print_Deriv = pd.concat([data_print_Forwards, data_print_Swaps], axis=0)
# Объединение таблиц вниз

data_print_Deriv_columns = {x:y for x,y in zip(data_print_Deriv.columns.tolist(), [[] for x in data_print_Deriv.columns.tolist()])}
data_print_Deriv_columns['Entity_group']=[]
data_print_Deriv_SUEK = pd.DataFrame(data_print_Deriv_columns)
data_print_Deriv_Ech = pd.DataFrame(data_print_Deriv_columns)
for i in ['SUEK', 'EuroChem']:
    data_print_Deriv_group = data_print_Deriv[data_print_Deriv.holdingEntity == i].reset_index(drop=True)
    if len(data_print_Deriv_group) == 0:
        continue   
    if i == 'SUEK':
        data_print_Deriv_group = data_print_Deriv_group.rename(columns={'entity': 'pre_entity'})
        data_print_Deriv_group['entity'] = merge_Mapping(Data_for_loop, col='pre_entity')
        
    data_print_Deriv_group['Entity_group'] = merge_SalesUnits(data_print_Deriv_group, col='entity', merge_col='ocpSegment').fillna('External')
    data_print_Deriv_group = data_print_Deriv_group[['entity','Entity_group','holdingEntity','Company',\
        'counterparty','dealNo','payFXCurrency','payFaceValue',\
        'payFaceValue_in_USD','Settlement','Notional_Amount_(USD)',\
        'Unnamed','maturityDate','Days','Period','source']]
    if i == 'SUEK':
        data_print_Deriv_SUEK = data_print_Deriv_group
    if i == 'EuroChem':
        data_print_Deriv_Ech = data_print_Deriv_group

C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_3232\388960480.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Data_for_loop.loc[len(Data_for_loop)] = row_list[:4] + row_list[6:8] + row_list[6:]
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_3232\388960480.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  Data_for_loop.loc[len(Data_for_loop)] = row_list[:4] + row_list[6:8] + row_list[6:]
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_3232\388960480.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.py

data_Swaps_export is empty


C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:93: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)
C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '['NAMSALES' 'NAMSALES' 'NAMSALES' 'NAMSALES' 'NAMSALES' 'NAMSALES'
 'NAMSALES' 'NAMSALES' 'NAMSALES' 'NAMSALES' 'NAMSALES' 'NAMSALES'
 'NAMSALES' 'NAMSALES' 'NAMSALES' 'NAMSALES' 'NAMSALES' 'NAMSALES'
 'NAMSALES' 'NAMSALES' 'NAMSALES' 'NAMSALES' 'NAMSALES' 'NAMSALES'
 'NAMSALES' 'NAMSALES' 'NAMSALES' 'NAMSALES' 'SAMSALES' 'SAMSALES']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  merge_data.loc[merge_data[f'{id_col}_merge'] != 'External', '

Запись Deriv в файл:

In [23]:
Sheet_in_output_Deriv = 'Deriv'
Output_Deriv_SUEK = "_".join([str(date.today()), 'SUEK_quantum_Deriv', f'{Mounth}.xlsx'])
Output_Deriv_Ech = "_".join([str(date.today()), 'Ech_quantum_Deriv', f'{Mounth}.xlsx'])
if excel_tofolder_on_Z == True:
    Output_path = 'z:\\Anna_Klimova\\OCP\\Archive\\'
    Output_Deriv_SUEK = Output_path + Output_Deriv_SUEK
    Output_Deriv_Ech = Output_path + Output_Deriv_Ech

if print_Deriv == True:
    if len(data_print_Deriv_SUEK) != 0:
        data_print_Deriv_SUEK.to_excel(Output_Deriv_SUEK, sheet_name=Sheet_in_output_Deriv, index=False)
    if len(data_print_Deriv_Ech) != 0:
        data_print_Deriv_Ech.to_excel(Output_Deriv_Ech, sheet_name=Sheet_in_output_Deriv, index=False)

In [24]:
assert 2==1

AssertionError: 

Сверка форвардов с новой бд

In [ ]:
# query = f"""select * from "RISKACCESS"."unifiedOcp" 
# where "reportDate" = TO_DATE('{Today}', 'DD/MM/YY')
# and "sourceSystem" = 'QUANTUM'
# """
# data = export_from_RISKCUSTOM(query)
# data_Der = data[data.accountType == 'Derivative']
# unifiedOcp_dealNo_list = data_Der.recordId.unique().tolist()

# query = f"""select * from "RISKACCESS"."quantumForwards" """
# data_Forwards_export = export_from_RISKCUSTOM(query)
# data_list = data_Forwards_export[data_Forwards_export.positionDate > '2024-03-01'].positionDate.dt.date.astype(str).unique().tolist()

# date_diff_dict = {}

# for date_i in data_list:
#     query = f"""select * from "RISKACCESS"."quantumForwards" where "positionDate" = TO_DATE('{date_i}', 'YYYY-MM-DD')"""
#     data_Forwards_export = export_from_RISKCUSTOM(query)
#     dealNo_intersect_list = list(set(data_Forwards_export['dealNo']).intersection(set(data_Debt_export['dealNo'])))
#     data_work_Forwards = data_Forwards_export[~(data_Forwards_export.dealNo.isin(dealNo_intersect_list))]

#     Forwars_dealNo_list = data_work_Forwards.dealNo.astype(str).unique().tolist()
#     deal_No_intersect_list = [i for i in Forwars_dealNo_list if i in unifiedOcp_dealNo_list]
#     diff_F_inter = len(Forwars_dealNo_list) - len(deal_No_intersect_list)
#     diff_U_inter = len(unifiedOcp_dealNo_list) - len(deal_No_intersect_list)

#     date_diff_dict[date_i] = {'diff_F': diff_F_inter, 'diff_U': diff_U_inter}

# diff_df = pd.DataFrame(date_diff_dict).transpose()
# diff_df[(diff_df.diff_F == diff_df.diff_F.min()) & (diff_df.diff_U == diff_df.diff_U.min())]

In [ ]:
# query = f"""select * from "RISKACCESS"."quantumForwards" where "positionDate" = TO_DATE('2024-04-03', 'YYYY-MM-DD')"""
# data_Forwards_export = export_from_RISKCUSTOM(query)
# dealNo_intersect_list = list(set(data_Forwards_export['dealNo']).intersection(set(data_Debt_export['dealNo'])))
# data_work_Forwards = data_Forwards_export[~(data_Forwards_export.dealNo.isin(dealNo_intersect_list))]

# data_Forwards_piece = data_work_Forwards[['dealNo', 'payFXCurrency', 'payFaceValue']].reset_index(drop=True)
# data_Forwards_piece.dealNo = data_Forwards_piece.dealNo.astype(str)
# data_Der_piece = data_Der[['recordId', 'effectiveCurrency', 'volumeEffectiveCurrency']].reset_index(drop=True)
# data_merge = data_Der_piece.merge(data_Forwards_piece, how='outer', left_on='recordId', right_on='dealNo')
# data_merge = data_merge[data_merge.effectiveCurrency == data_merge.payFXCurrency]
# data_merge

Тестовая БД:

In [ ]:
# query = f"""
# select * from "unifiedOcp"
# where "sourceSystem" = 'QUANTUM'
# and "reportDate" = TO_DATE('{Today}', 'DD/MM/YY')
# """
# data = export_from_WHWEEK(query)

query = f"""select * from "RISKACCESS"."unifiedOcp" 
where "reportDate" = TO_DATE('{Today}', 'DD/MM/YY')
and "sourceSystem" = 'QUANTUM'
"""
data = export_from_RISKCUSTOM(query)

group = 0
data_list = [f'Слева-направо Вы ыидите выгрузку из QUANTUM и соответсвюущие ей строки из WHFLEX на {Today}', '']

data.loc[data.holding.isna(),'holding'] = 'SUEK'
for group in data.holding.unique().tolist():
    data_work = data.loc[data.holding == group ,['##origin_name', 'holding', 'buCode', 'ocpSegment', 'counterpartyName', 'introgroupCounterpartyCode', 'counterpartySegment', 'effectiveCurrency', 'maturityDateFrom', 'volumeEffectiveCurrency']].reset_index(drop=True)

    data_work.loc[data_work.counterpartyName.str.contains('(EC)'), 'counterpartyName'] = data_work.loc[data_work.counterpartyName.str.contains('(EC)'), 'counterpartyName'].str[:-5] # Обрезка " (EC)" в каждой строке

    if group == 'SUEK':
        data_work['CompCode'] = merge_Mapping(data_work, 'buCode')
        data_work = data_work.rename(columns={'buCode': 'pre_buCode', 'CompCode':'buCode'})
        
        data_test = data_Debt_work_SUEK[['##origin_name', 'holding', 'entity', 'Entity_group', 'counterparty', 'CompCode', 'Counterparty_Group', 'eventCurrency','actionDate', 'amount']]
        data_der = data_print_Deriv_SUEK[['source', 'holdingEntity', 'entity', 'Entity_group', 'counterparty', 'payFXCurrency', 'maturityDate', 'payFaceValue']]

    if group == 'EUROCHEM':
        data_test = data_Debt_work_ECH[['##origin_name', 'holding', 'entity', 'Entity_group', 'counterparty', 'CompCode', 'Counterparty_Group', 'eventCurrency','actionDate', 'amount']]
        data_der = data_print_Deriv_Ech[['source', 'holdingEntity', 'entity', 'Entity_group', 'counterparty', 'payFXCurrency', 'maturityDate', 'payFaceValue']]

    data_work = data_work.fillna('External')
    data_work = data_work.groupby(['##origin_name', 'holding', 'buCode', 'ocpSegment', 'counterpartyName', 'introgroupCounterpartyCode', 'counterpartySegment', 'effectiveCurrency', 'maturityDateFrom'], as_index=False).agg({'volumeEffectiveCurrency': 'sum'})
    data_work = concat_columns(df=data_work, columns=['buCode', 'counterpartyName', 'effectiveCurrency', 'maturityDateFrom']).reset_index(drop=True)

    # Работа с деривативами 
    if len(data_der) != 0:
        data_der.loc[data_der.counterparty.str.contains('(EC)'), 'counterparty'] = data_der.loc[data_der.counterparty.str.contains('(EC)'), 'counterparty'].str[:-5] # Обрезка " (EC)" в каждой строке
        data_der['CompCode'] = merge_Mapping(data_der, col='counterparty')
        data_der['Counterparty_Group'] = merge_SalesUnits(data_der, col='CompCode', merge_col='ocpSegment')
        data_der = data_der.fillna('External')
        data_der = data_der.rename(columns={'source':'##origin_name', 'holdingEntity':'holding', 'payFXCurrency':'eventCurrency', 'maturityDate':'actionDate', 'payFaceValue':'amount'})
        # data_der = data_der.groupby(['##origin_name', 'holding', 'entity', 'Entity_group', 'counterparty', 'CompCode', 'Counterparty_Group', 'eventCurrency','actionDate'], as_index=False).agg({'amount': 'sum'})
    else:
        data_der = pd.DataFrame()
    
    data_test =  pd.concat([data_test, data_der], axis=0)
    data_test = data_test.fillna('External')
    data_test = data_test.groupby(['##origin_name', 'holding', 'entity', 'Entity_group', 'counterparty', 'CompCode', 'Counterparty_Group', 'eventCurrency','actionDate'], as_index=False).agg({'amount': 'sum'})
    data_test = concat_columns(data_test, ['entity', 'counterparty', 'eventCurrency', 'actionDate']).reset_index(drop=True)

    data_merge = data_test.merge(data_work, how='outer', left_on='concat_columns', right_on='concat_columns')
    
    # Сравнение величин 
    data_merge[['volumeEffectiveCurrency', 'amount']] = data_merge[['volumeEffectiveCurrency', 'amount']].fillna(0).astype('int64')
    data_merge['Amount_is_equal'] = data_merge.amount.abs() == data_merge.volumeEffectiveCurrency.abs()

    col_dict = {'Entity_group':'ocpSegment', 'CompCode':'introgroupCounterpartyCode', 'Counterparty_Group':'counterpartySegment', 'holding_x':'holding_y'}
    data_merge.holding_y = data_merge.holding_y.replace({'EUROCHEM':'EuroChem'})

    for key in list(col_dict.keys()):
        data_merge[f'{key}_is_equal'] = np.NaN
        data_merge_key_notna = data_merge[(~data_merge[key].isna()) & (~data_merge[col_dict[key]].isna())]
        data_merge_key_notna[f'{key}_is_equal'] = data_merge_key_notna[key] == data_merge_key_notna[col_dict[key]]
        data_merge[(~data_merge[key].isna()) & (~data_merge[col_dict[key]].isna())] = data_merge_key_notna

    is_equal_cols_list = data_merge.columns[data_merge.columns.str.contains('is_equal')].tolist()
    data_list.append(f'{group} points:')
    for col in is_equal_cols_list:
        if False in data_merge[col].unique().tolist():
            data_list.append(f'{col}: ALARM')
        else:
            data_list.append(f'{col}: ok')
    data_list.append('')

    if group == 'SUEK':
        data_merge_SUEK = data_merge
    if group == 'EUROCHEM':
        data_merge_Ech = data_merge

C:\Users\KlimovaAnnaA\Documents\MyFiles\Projects\OCP\Defs.py:89: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_export = pd.read_sql(query, con=connection)
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_15688\1164580333.py:21: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  data_work.loc[data_work.counterpartyName.str.contains('(EC)'), 'counterpartyName'] = data_work.loc[data_work.counterpartyName.str.contains('(EC)'), 'counterpartyName'].str[:-5] # Обрезка " (EC)" в каждой строке
C:\Users\KlimovaAnnaA\AppData\Local\Temp\ipykernel_15688\1164580333.py:66: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

In [ ]:
Output_BD = "_".join([str(date.today()), 'QUANTUM_to_WHFLEX', f'{Mounth}.xlsx'])
if print_BD == True:
    write_data = {'data': data_list}
    df = pd.DataFrame(write_data)
    df.to_excel(Output_BD, sheet_name='Iinstruction', index=False, header=False)
    
    new_list(data_merge_SUEK, Output_BD, sheet_name='SUEK', index=True)
    new_list(data_merge_Ech, Output_BD, sheet_name='Ech', index=True)